# Lab 5: Wide and Deep Networks

## Preparation

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Read the data
data = 'healthcare-dataset-stroke-data.csv'
df = pd.read_csv(data)
print(df.shape)
df.head()

(5110, 12)


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [2]:
# check for duplicates
df.duplicated().sum()

0

In [3]:
# drop id column
df = df.drop(['id'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 439.3+ KB


The data is organized as follows:

|Variable | Description|
|---------|------------|
|gender| Male/Female|
|age| continuous|
|hypertension| 0=No, 1=Yes|
|heart_disease| 0=No, 1=Yes|
|ever_married| No/Yes|
|work_type| children, Govt_jov, Never_worked, Private, Self-employed|
|Residence_type| Rural/Urban|
|avg_glucose_level| continuous|
|bmi| continuous|
|smoking_status| formerly smoked, never smoked, smokes, Unknown|
|`stroke`| 0=No, 1=Yes|

In [4]:
# Check for missing values
print(df.isnull().sum())

gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64


BMI is the only column with missing values. 201/5110 ~= 3.93% of the rows don't have BMI; that's not a huge percentage, so we will use KNN imputation to fill in these values.

As a note, a large portion of the smoking values - almost 30% - are "unknown." We debated whether or not to treat this as essentially a NaN and drop or impute, but we decided to leave these rows in in case we discover later that the "unknown" values skew towards strokes or not.

In [5]:
from sklearn.impute import KNNImputer
import copy

# Use k = 3 nearest neighbors based on age, bmi, and average glucose level
knn_obj = KNNImputer(n_neighbors=3) 
attributes_to_impute_with = ['age', 'bmi', 'avg_glucose_level']
temp = df[attributes_to_impute_with].to_numpy()
temp_imputed = knn_obj.fit_transform(temp)
df_imputed = copy.deepcopy(df)
df_imputed[attributes_to_impute_with] = temp_imputed
df_imputed.dropna(inplace=True)
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                5110 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 439.3+ KB


In [6]:
import tensorflow as tf
import keras

print(tf.__version__)
print(keras.__version__)

2.13.0
2.13.1


In [7]:
from tensorflow.keras.layers import Dense, Input, Activation
from tensorflow.keras.layers import Embedding, Flatten, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

In [8]:
# create a tensorflow dataset
batch_size = 64

def df_to_dataset(df, shuffle=True, batch_size=batch_size):
    df = df.copy()
    labels = df.pop('stroke')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

ds = df_to_dataset(df_imputed)

In [9]:
# print out the data type of each feature
for feature_batch, label_batch in ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    # print feature datatype
    print('Feature shape for age is:', feature_batch['age'].shape)
    print('Max:', max(feature_batch['age']))

Every feature: ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status']
Feature shape for age is: (64,)
Max: tf.Tensor(82.0, shape=(), dtype=float64)


#### Cross-Product Features

Our categorical features are gender, hypertension, heart_disease, ever_married, work_type, and Residence_type. hypertension and heart_disease are already encoded as integers; the other five are strings. When building a keras feature space, we need to consider which categories make sense to combine into cross-product features. After working with this dataset in lab 1, we believe the following combinations should be made:

#### TODO: We are discretizing age so we can include in crossing, correct? If not, just do hypertension and heart disease.

- **age, hypertension, heart_disease:** Age is the highest correlating risk factor for stroke. The median age for a stroke patient in our dataset is 70. This means that anyone at that age or above should certainly be screened. Additionally, hypertension and heart disease are serious underlying health conditions. They both serve individually as strong risk factors for stroke, and the combination of the two is an even greater risk factor. Age also correlates strongly with hypertension and heart disease, so we believe it makes sense to include them in a cross-product with age. In lab 1, we found that an alarming 25% of patients with both conditions suffered from a stroke. We definitely want to memorize this combination.
- **ever_married, Residence_type, work_type:** The three "lifestyle" attributes make sense to combine. The lifestyle attributes likely contribute to a person's stress levels. [According to the Heart and Stroke Foundation of Canada](https://www.heartandstroke.ca/healthy-living/reduce-stress/stress-basics), stress levels increase the likelihood for strokes and increase the underlying risk factors for strokes, such as hypertension. Again, in lab 1, we saw that certain combinations of the lifestyle attributes resulted in positive stroke diagnoses at a rate higher than the general population (which is about 5%). 


In [11]:
from tensorflow.keras.utils import FeatureSpace


# Create the feature space with preprocessing steps applied and crossed features
feature_space = FeatureSpace(
    features={

        # Categorical features encoded as string
        'gender': FeatureSpace.string_categorical(num_oov_indices=0),
        'ever_married': FeatureSpace.string_categorical(num_oov_indices=0),
        'work_type': FeatureSpace.string_categorical(num_oov_indices=0),
        'Residence_type': FeatureSpace.string_categorical(num_oov_indices=0),
        'smoking_status': FeatureSpace.string_categorical(num_oov_indices=0),

        # Numerical features
        'age': FeatureSpace.float_discretized(num_bins=10),

        # Numerical features to normalize
        'avg_glucose_level': FeatureSpace.float_normalized(),
        'bmi': FeatureSpace.float_normalized(),

        # Categorical features encoded as integers
        'hypertension': FeatureSpace.integer_categorical(num_oov_indices=0),
        'heart_disease': FeatureSpace.integer_categorical(num_oov_indices=0),
    },
    crosses=[
        # Is 10 a crossing dimension for age if there are 10 bins?
        # Can we do this while initializing the feature space?
        # Or Should age be excluded?
        FeatureSpace.cross(feature_names=('age', 'hypertension', 'heart_disease'), crossing_dim=10*2*2), 
        FeatureSpace.cross(feature_names=('ever_married', 'Residence_type', 'work_type'), crossing_dim=2*2*5), 
    ],
    output_mode='concat',
)


# Create a dataset with the feature space
ds_no_labels = ds.map(lambda x, _: x)
feature_space.adapt(ds_no_labels)
preprocessed_ds = ds.map(lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE)
preprocessed_ds = preprocessed_ds.prefetch(tf.data.AUTOTUNE)

### Evaluation Metric

Our evaluation metric of choice will be F-beta with a larger beta value. For stroke risk classification, we want to minimize the number of false negatives as much as possible. Consider the following two scenarios:

- **False positive:** a patient is at low risk, but we classify them as high risk. This means that a patient may undergo unecessary treatment, expenses, and stress if they are screened to visit a doctor, believing they are at high risk for a stroke. However, ultimately the patient is unlikley to suffer from a stroke, so they are still safer despite the false positive.
- **False negative:** a patient is at high risk, but we classify them as low risk. This means that a patient falsely believes they are at low risk for a stroke. In this case, key warning signs could go undetected or ignored, and a patient may suffer a stroke without expecting one at all. They would require emergency care, and potentially suffer the most severe consequence possible: death.

False negatives and false positives are both bad; however, false negatives are clearly worse. As such, we need an evaluation metric that will tell us if our model is good at avoiding false negatives, while still doing our best to avoid false positives. F-beta can accomplish that for us; if we make beta greater than 1, the addition of the beta term in the denominator weighs recall more importantly than precision. This is what we're looking for. 

#### Data Divison Method

When it comes to dividing our data into training and testing, we feel that **stratified k-fold cross validation** makes the most sense. There is a large class imbalance present in our dataset; only approximately 5% (250/5110) of patients actually suffered a stroke, and 95% of patients did not. Using stratified k-fold cross validation will allow for each fold to mirror the original class imbalance present in the dataset. If we were to use non-stratified cross validation or shuffle splits, given the large class imbalance, it's very likely that getting randomized train and test folds could result in 100% non-stroke patients in the fold, which would make for a poor model when it comes to testing. If the model hasn't seen positive stroke patients, it won't know how to classify them. We want to make sure that each fold sees at least a few positive stroke patients. Additionally, given that we only have a moderately sized dataset at 5000 instances, dividing our dataset into training and testing folds will allow us to reuse the data multiple times, instead of trying to generate synthetic samples which could induce noise. In addition, we can take metrics on every iteration of the cross validation process, and then average them or calculate the standard deviation to understand how our model performs across different folds and bound our expectations of it. 

This is a realistic method to use because in many real-world medical applications, including stroke risk prediction, datasets tend to be imbalanced, with a smaller number of high-risk cases compared to low-risk cases. Using stratified k-fold cross validation to mirror this reality allows us to view how our model would perform on these underrepresented, high-risk cases.


In [12]:
dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

# Build the model
x = keras.layers.Dense(64, activation='relu')(encoded_features)
x = keras.layers.Dense(32, activation='relu')(x)
predictions = keras.layers.Dense(1, activation='sigmoid')(x)

# this expects features that are already transformed
training_model = keras.Model(inputs=encoded_features, outputs=predictions)
training_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# this expects features that are not transformed
inference_model = keras.Model(inputs=dict_inputs, outputs=predictions)
inference_model.compile(loss='binary_crossentropy', metrics=['accuracy'])

inference_model.summary()

trainin

plot_model(training_model, show_shapes=True, show_layer_names=True, to_file='model.png',
           rankdir='LR', expand_nested=False, dpi=96)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Residence_type (InputLayer  [(None, 1)]                  0         []                            
 )                                                                                                
                                                                                                  
 age (InputLayer)            [(None, 1)]                  0         []                            
                                                                                                  
 ever_married (InputLayer)   [(None, 1)]                  0         []                            
                                                                                                  
 heart_disease (InputLayer)  [(None, 1)]                  0         []                      

NameError: name 'trainin' is not defined

In [16]:
# TODO: Use this methodology when actually running the model.

# from sklearn.model_selection import cross_val_score, StratifiedKFold
# from sklearn.metrics import fbeta_score

# clf = training_model
# cv = StratifiedKFold(n_splits=10)
# scorer = make_scorer(fbeta_score, beta=2)
# per_fold_eval_criteria = cross_val_score(estimator=clf, X=preprocessed_ds, y=y, cv=cv, scoring=scorer)